**Задание 1**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

df = pd.read_excel('orders.xlsx')

numerical = [
    'версия погрузки',
    'Количество вагонов',
    'Контрибуция', 
    'Длительность жизни',
    'Время до отправки',
    'Количество редакций после отправки',
    'Количество отмен', 
    'Количество переносов',
    'Среднее время между редакциями', 
    'Длительность афтершока']

df_rule = df['Дата отправления']>='2023-09-01'
X_train = df[~df_rule].drop('Результат', axis = 1)
X_train = X_train[numerical]
y_train = df['Результат'][~df_rule].map({0: -1, 1: 1})
X_test = df[df_rule].drop('Результат', axis = 1)
X_test = X_test[numerical]
y_test = df['Результат'][df_rule].map({0: -1, 1: 1})

In [2]:
class AdaBoost_custom:
    def __init__(self, n_estimators=5, base_function=LogisticRegression()):
        self.n_estimators = n_estimators
        self.base_estimator = base_function
        self.alphas = []
        self.models = []

    def fit(self, X, y):
        l = len(X)
        w = np.ones(l) / l
        for _ in range(self.n_estimators):
            b = self.base_estimator.__class__()  # Создание нового экземпляра классификатора
            b.fit(X, y, sample_weight=w)
            y_pred = b.predict(X)
            P = np.sum(w * (y_pred == y)) / np.sum(w)
            alpha = 0.5 * np.log(P / (1 - P))
            w *= np.exp(-alpha * y * y_pred)
            w /= np.sum(w)
            self.models.append(b)
            self.alphas.append(alpha)

    def predict(self, X):
        return np.sign(sum(alpha * model.predict(X) for alpha, model in zip(self.alphas, self.models)))

    def fit_predict(self, X, y):
        self.fit(X, y)
        return self.predict(X)

In [3]:
import warnings
warnings.filterwarnings("ignore")

model = AdaBoost_custom(n_estimators=10, base_function=LogisticRegression())
y_pred = model.fit_predict(X_train, y_train)

In [4]:
y_pred

array([-1.,  1.,  1., ...,  1., -1.,  1.])

**Задание 2**

In [5]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

classifiers = {
    "LogisticRegression": {
        "estimator": [LogisticRegression()],
        "estimator__C": [0.01, 0.1]
    },
    "DecisionTree": {
        "estimator": [DecisionTreeClassifier()],
        "estimator__max_depth": [2, 4, 6]
    },
    "RandomForest": {
        "estimator": [RandomForestClassifier()],
        "estimator__max_depth": [2, 4, 6]
    }
}

In [6]:
ada_params = {
    "n_estimators": [10, 50, 100]}

In [7]:
best_classifier = None
best_params = None
best_score = 0

for clf_name, param_grid in classifiers.items():
    full_param_grid = {**param_grid, **ada_params}
    grid_search = GridSearchCV(AdaBoostClassifier(), full_param_grid, cv=5, scoring="accuracy", n_jobs=-1)
    grid_search.fit(X_train, y_train)
    
    if grid_search.best_score_ > best_score:
        best_score = grid_search.best_score_
        best_classifier = grid_search.best_estimator_
        best_params = grid_search.best_params_

In [8]:
print("Классификатор:", best_classifier.estimator)
print("Немногочисленные параметры:", best_params)
print("Точность:", best_score)

Классификатор: RandomForestClassifier(max_depth=4)
Немногочисленные параметры: {'estimator': RandomForestClassifier(), 'estimator__max_depth': 4, 'n_estimators': 100}
Точность: 0.8961541889483066


**Задание 3**

In [9]:
from sklearn.inspection import permutation_importance

In [10]:
result = permutation_importance(best_classifier, X_test, y_test, n_repeats=10, random_state=42, scoring="accuracy")

In [11]:
importance_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": result.importances_mean
})

top_10_df = importance_df.sort_values(by="Importance", ascending=False).head(10).reset_index(drop=True)
top_10_df

,Feature,Importance
0,Количество отмен,0.252577
1,версия погрузки,0.224485
2,Количество редакций после отправки,0.171649
3,Длительность афтершока,0.103866
4,Количество переносов,0.066495
5,Среднее время между редакциями,0.057216
6,Длительность жизни,0.027062
7,Контрибуция,0.017010
8,Время до отправки,0.010309
9,Количество вагонов,-0.004124


**Выводы**

Лучшим классификатором тут стал случайный лес. Спойлер - если его не брать в гридсерч, то лучше всего DecisionTree. Параметры, которые больше всего влияют на результат - кол-во отмен, версия погрузки и кол-во редакций. Также можно сделать вывод, что мой ноубук очень не любит гридсерч и чуть ли не умирает от него((